# Introduction to RSA (in Python)

This tutorial was adopted from the first chapter of the ProbLang textbook: https://www.problang.org/chapters/01-introduction.html

The primary difference is that everything is implemented in Python. Probabilistic programming languages like WebPPL give us a nice abstraction for inferring and sampling from distributions, but it turns out that we don't actually need to do anything fancy for simple RSA. We can compute posteriors by enumeration, a.k.a. just applying the exact mathematical formulas.

If this is your first time with RSA, I would highly recommend reading the linked chapter. This is mostly just an exercise in translating to Python.

In [1]:
import numpy as np
import pandas as pd

## Define objects and utterances

First, we define the objects in the world (which we likewise refer to as the set of _world states_), as well as the possible utterances.

In [2]:
objects = [
            {"color": "blue", "shape": "square", "string": "blue square"},
            {"color": "blue", "shape": "circle", "string": "blue circle"},
            {"color": "green", "shape": "square", "string": "green square"}
]
utterances = ["blue", "green", "square", "circle"]

Next, we define a meaning function that uses simple Boolean semantics: it returns True if the utterance is true of the object, and false otherwise.

In [3]:
def meaning(obj, utt):
    return (utt == obj["color"]) or (utt == obj["shape"])

Run the following cell to see how the meaning function works:

In [4]:
meaning({"color": "blue", "shape": "square"}, "circle")

False

## Literal listener

The literal listener is then defined via a function that maps utterances to a probability distribution over world states. The function is defined as follows:

$$P_{L_0}(s∣u) \propto [[u]](s) \cdot P(s)$$

where $[[u]](s)$ is the meaning function.

We'll define a helper function `normalize_rows` that will be useful for successive calculations.

In [5]:
def normalize_rows(matrix):
    """
    Helper function that normalize probabilities across rows to sum to 1
    """
    totals = np.sum(matrix, axis=1)
    return matrix / totals[:, np.newaxis]

In [6]:
def literal_listener(utt):
    """
    Simulate a literal listener
    
    Arguments:
    utt: string that represents what is heard by the listener
    
    Return:
    df: pd.DataFrame of object probabilities for all possible utterances
    probs: pd.Series of the object probabilities associated with the given utterance
    """
    # generate the matrix of utterances x world states
    all_counts = np.zeros(shape=(len(utterances), len(objects)))
    for i in range(len(utterances)):
        for j in range(len(objects)):
            curr_utt = utterances[i]
            curr_obj = objects[j]

            if meaning(curr_obj, curr_utt): all_counts[i, j] = 1
            # if I wanted to incorporate a prior I would do it here
                
    data = normalize_rows(all_counts)
    df_cols = [obj["string"] for obj in objects]
    df = pd.DataFrame(data, columns=df_cols, index=utterances)
    return df, df.loc[utt]

In [10]:
df_l0, probs_l0 = literal_listener("square")
df_l0

,blue square,blue circle,green square
blue,0.5,0.5,0.0
green,0.0,0.0,1.0
square,0.5,0.0,0.5
circle,0.0,1.0,0.0


In [11]:
probs_l0

blue square     0.5
blue circle     0.0
green square    0.5
Name: square, dtype: float64

## Pragmatic speaker

Now we take into account a speaker's utility function

$$U_{S_1}(u;s) = \log L_0(s∣u) − C(u)$$

as part of our expression for the pragmatic speaker, which is a function that maps probabilities over world states to utterances. Formally, it is defined as

$$P_{S_1}(u∣s) \propto \exp(\alpha \cdot U_{S_1}(u;s)),$$
which expands to
$$P_{S_1}(u∣s) \propto \exp(\alpha \cdot (\log L_0(s∣u) − C(u))).$$

Let's ignore $C(u)$ for now.

In [12]:
def pragmatic_speaker(obj, alpha=1):
    """
    Simulate the pragmatic speaker
    
    Arguments:
    obj: dict to represent the object in the world that the speaker wishes to refer to
    alpha: float for speaker optimality (default set to 1)
    
    Return:
    df: pd.DataFrame of utterance probabilities for all possible objects in the world
    probs: pd.Series of utterance probabilities for the specified object
    """
    all_vals = []
    for curr_utt in utterances:
        _, probs = literal_listener(curr_utt)
        utility = np.array(probs)
        val = np.exp(alpha * np.log(utility))
        all_vals.append(val)
        
    data = normalize_rows(np.array(all_vals).T)
    df_idx = [obj["string"] for obj in objects]
    df = pd.DataFrame(data, columns=utterances, index=df_idx)
    return df, df.loc[obj["string"]]

In [13]:
df_s1, probs_s1 = pragmatic_speaker({"color": "blue", "shape": "square", "string": "blue square"})
df_s1

/var/folders/wh/tk6j12kd7jq5djbxfn93wjdm0000gn/T/ipykernel_14878/3154548776.py:17: RuntimeWarning: divide by zero encountered in log
  val = np.exp(alpha * np.log(utility))


,blue,green,square,circle
blue square,0.500000,0.000000,0.500000,0.000000
blue circle,0.333333,0.000000,0.000000,0.666667
green square,0.000000,0.666667,0.333333,0.000000


In [14]:
probs_s1

blue      0.5
green     0.0
square    0.5
circle    0.0
Name: blue square, dtype: float64

## Pragmatic listener

Finally, we can define the pragmatic listener as follows:

$$P_{L_1}(s \vert u) \propto P_{S_1}(u \vert s) \cdot P(s)$$

In [26]:
def pragmatic_listener(utt):
    all_vals = []
    for curr_obj in objects:
        _, probs = pragmatic_speaker(curr_obj)
        all_vals.append(probs)

    data = normalize_rows(np.array(all_vals).T)
    df_cols = [obj["string"] for obj in objects]
    df = pd.DataFrame(data, columns=df_cols, index=utterances)
    return df, df.loc[utt]

In [27]:
df_l1, probs_l1 = pragmatic_listener('blue')
df_l1

/var/folders/wh/tk6j12kd7jq5djbxfn93wjdm0000gn/T/ipykernel_14878/3154548776.py:17: RuntimeWarning: divide by zero encountered in log
  val = np.exp(alpha * np.log(utility))


,blue square,blue circle,green square
blue,0.6,0.4,0.0
green,0.0,0.0,1.0
square,0.6,0.0,0.4
circle,0.0,1.0,0.0


In [28]:
probs_l1

blue square     0.6
blue circle     0.4
green square    0.0
Name: blue, dtype: float64

So this model predicts that a pragmatic listener who hears the utterance "blue" is more likely to think that the intended referent is the blue square.